<a href="https://colab.research.google.com/github/dariiazorii/OID/blob/main/lab2_oid/lab_2_oid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 3. Монтування Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# 4. Створення SparkSession та робота
print("4/4: Створення SparkSession та перетворення...")
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("LAB_1") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()


csv_file = "/content/drive/MyDrive/Colab Notebooks/oid/2019-Oct.csv"
parquet_output = "/content/2019_Oct_parquet"

df = spark.read.csv(csv_file, header=True, inferSchema=True)
df.write.parquet(parquet_output, mode="overwrite")
print("Перетворення CSV у Parquet успішно завершено.")
print("PySpark встановлено та налаштовано. Можна створювати SparkSession.")

4/4: Створення SparkSession та перетворення...
Перетворення CSV у Parquet успішно завершено.
PySpark встановлено та налаштовано. Можна створювати SparkSession.


In [ ]:
from pyspark.sql.functions import col

# 1. Product_Dim (Зірка): Включає category_code (денормалізовано)
product_dim_star = df.select("product_id", "category_id", "category_code", "brand") \
                     .distinct()

# 2. User_Dim (Обидві схеми)
user_dim = df.select("user_id").distinct()

# 3. Sales_Fact (Обидві схеми)
sales_fact = df.select("event_time", "event_type", "price", "product_id", "user_id")

# Збереження таблиць Зірки
product_dim_star.write.parquet("/content/star_product_dim", mode="overwrite")
user_dim.write.parquet("/content/star_user_dim", mode="overwrite")
sales_fact.write.parquet("/content/sales_fact", mode="overwrite")

In [ ]:
# 1. Category_Dim (Сніжинка): Нова таблиця вимірів (нормалізація)
category_dim = df.select("category_id", "category_code").distinct()

# 2. Product_Dim (Сніжинка): НЕ включає category_code (тільки ключ)
product_dim_snowflake = df.select("product_id", "category_id", "brand") \
                          .distinct()

# Sales_Fact та User_Dim залишаються без змін (використовуємо вже створені)

# Збереження таблиць Сніжинки
category_dim.write.parquet("/content/snowflake_category_dim", mode="overwrite")
product_dim_snowflake.write.parquet("/content/snowflake_product_dim", mode="overwrite")

In [ ]:
import time
from pyspark.sql.functions import count, col

# ЗІРКА (Таблиці: sales_fact, product_dim_star)
fact_star = spark.read.parquet("/content/sales_fact")
dim_product_star = spark.read.parquet("/content/star_product_dim")

# СНІЖИНКА (Таблиці: sales_fact, product_dim_snowflake, category_dim)
fact_snowflake = fact_star
dim_product_snow = spark.read.parquet("/content/snowflake_product_dim")
dim_category_snow = spark.read.parquet("/content/snowflake_category_dim")

print("Таблиці завантажено.")

Таблиці завантажено.


In [ ]:
star_start = time.time()

# Запит ЗІРКИ: Потрібен ОДИН JOIN між Sales_Fact та Product_Dim_Star
result_star = fact_star.filter(col("event_type") == "purchase") \
    .join(dim_product_star, "product_id") \
    .groupBy("category_code") \
    .agg(count("*").alias("total_purchases"))

# Примусове виконання (використовуємо noop для запису без збереження)
result_star.write.format("noop").mode("overwrite").save()
time_star = time.time() - star_start
print(f" Схема «Зірка» (1 JOIN): {time_star:.2f} сек.")

 Схема «Зірка» (1 JOIN): 10.80 сек.


In [ ]:
snowflake_start = time.time()

# Запит СНІЖИНКИ: Потрібно ДВА JOIN (Fact -> Product -> Category)
result_snowflake = fact_snowflake.filter(col("event_type") == "purchase") \
    .join(dim_product_snow, "product_id") \
    .join(dim_category_snow, "category_id") \
    .groupBy("category_code") \
    .agg(count("*").alias("total_purchases"))

# Примусове виконання
result_snowflake.write.format("noop").mode("overwrite").save()
time_snowflake = time.time() - snowflake_start
print(f"Схема «Сніжинка» (2 JOIN): {time_snowflake:.2f} сек.")

Схема «Сніжинка» (2 JOIN): 15.96 сек.


In [ ]:
print("\n--- Порівняння результатів ---")
print(f"Зірка (1 JOIN): {time_star:.2f} сек.")
print(f"Сніжинка (2 JOIN): {time_snowflake:.2f} сек.")

# Очікуваний результат: Схема «Зірка» має бути швидшою (менше JOIN'ів).


--- Порівняння результатів ---
Зірка (1 JOIN): 10.80 сек.
Сніжинка (2 JOIN): 15.96 сек.
